In [1]:
!pip install openmeteo-requests requests-cache retry-requests pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.4/681.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.8 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.9.23
    Uninstalling flatbuffers-25.9.23:
      Successfully uninstalled flatbuffers-25.9.23


In [2]:
!fusermount -u /content/drive || true
!umount /content/drive || true
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive')



fusermount: failed to unmount /content/drive: No such file or directory
umount: /content/drive: no mount point specified.
Mounted at /content/drive


In [4]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd
import os

In [25]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 5.9804,
    "longitude": 116.0735,  # Kota Kinabalu
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m","wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()#hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,#"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",#"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "KotaKinabalu"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    #"precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/KotaKinabalu1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 6.01°N 116.12°E
Elevation: 7.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/KotaKinabalu1.csv
 Data shape: (4018, 6)
    Country        Region        Date  Temperature  RelativeHumidity  \
1  Malaysia  KotaKinabalu  2014-01-01        26.40             88.79   
2  Malaysia  KotaKinabalu  2014-01-02        25.75             91.52   
3  Malaysia  KotaKinabalu  2014-01-03        25.06             93.17   
4  Malaysia  KotaKinabalu  2014-01-04        25.07             92.22   
5  Malaysia  KotaKinabalu  2014-01-05        25.48             91.98   

   WindSpeed  
1      16.49  
2       9.09  
3       7.29  
4      11.53  
5      15.07  


In [21]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 5.4141,
    "longitude": 100.3288,  # GeorgeTown = Point(5.4141, 100.3288, 10)
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "Penang"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/Penang1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 5.45°N 100.44°E
Elevation: 6.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/Penang1.csv
 Data shape: (4018, 6)
    Country  Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  Penang  2014-01-01        26.68             81.94       0.30
2  Malaysia  Penang  2014-01-02        26.73             84.18       2.20
3  Malaysia  Penang  2014-01-03        26.00             87.41       4.00
4  Malaysia  Penang  2014-01-04        26.32             84.67       1.80
5  Malaysia  Penang  2014-01-05        26.22             85.21       2.90


In [20]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 1.5533,
    "longitude": 110.3592,  #Kuching Sarawak = Point(1.5533, 110.3592, 27)
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m","wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "Kuching"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/Kuching1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 1.58°N 110.32°E
Elevation: 4.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/Kuching1.csv
 Data shape: (4018, 6)
    Country   Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  Kuching  2014-01-01        23.85             96.99       4.74
2  Malaysia  Kuching  2014-01-02        24.33             97.09       4.84
3  Malaysia  Kuching  2014-01-03        25.36             90.71       8.37
4  Malaysia  Kuching  2014-01-04        25.17             91.95      10.70
5  Malaysia  Kuching  2014-01-05        24.60             94.01       7.17


In [19]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 6.1210,
    "longitude": 100.3600,  #AlorSetar = Point(6.1210, 100.3600, 6)
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "AlorSetar"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/AlorSetar1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 6.15°N 100.38°E
Elevation: 4.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/AlorSetar1.csv
 Data shape: (4018, 6)
    Country     Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  AlorSetar  2014-01-01        26.11             76.95      21.52
2  Malaysia  AlorSetar  2014-01-02        26.48             77.45      20.62
3  Malaysia  AlorSetar  2014-01-03        26.40             78.64      17.37
4  Malaysia  AlorSetar  2014-01-04        26.38             79.88      15.41
5  Malaysia  AlorSetar  2014-01-05        26.32             77.90      14.90


In [26]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 6.1254,
    "longitude": 102.2386,  #KotaBharu = Point(6.1254, 102.2386, 5)
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "KotaBharu"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/KotaBharu1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 6.15°N 102.26°E
Elevation: 11.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/KotaBharu1.csv
 Data shape: (4018, 6)
    Country     Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  KotaBharu  2014-01-01        24.98             85.16      18.06
2  Malaysia  KotaBharu  2014-01-02        25.41             86.15      19.88
3  Malaysia  KotaBharu  2014-01-03        25.23             88.25      11.79
4  Malaysia  KotaBharu  2014-01-04        25.65             84.34      12.41
5  Malaysia  KotaBharu  2014-01-05        24.64             89.61      15.91


In [27]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 5.3290,
    "longitude": 103.1369,  #KualaTerengganu = Point(5.3290, 103.1369, 6)
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "KualaTerengganu"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/KualaTerengganu1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 5.31°N 103.10°E
Elevation: 12.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/KualaTerengganu1.csv
 Data shape: (4018, 6)
    Country           Region        Date  Temperature  RelativeHumidity  \
1  Malaysia  KualaTerengganu  2014-01-01        24.61             86.39   
2  Malaysia  KualaTerengganu  2014-01-02        24.95             86.69   
3  Malaysia  KualaTerengganu  2014-01-03        24.70             88.50   
4  Malaysia  KualaTerengganu  2014-01-04        25.27             83.85   
5  Malaysia  KualaTerengganu  2014-01-05        24.45             88.11   

   WindSpeed  
1      19.96  
2      16.92  
3      11.30  
4      16.26  
5      14.26  


In [28]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 3.152815,
    "longitude": 101.703651,  #KL  3.152815, 101.703651
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "KualaLumpur"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/KualaLumpur.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 3.13°N 101.69°E
Elevation: 89.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/KualaLumpur.csv
 Data shape: (4018, 6)
    Country       Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  KualaLumpur  2014-01-01        25.43             84.41       8.40
2  Malaysia  KualaLumpur  2014-01-02        25.58             86.57       8.09
3  Malaysia  KualaLumpur  2014-01-03        24.42             92.79       9.89
4  Malaysia  KualaLumpur  2014-01-04        24.23             93.34      11.18
5  Malaysia  KualaLumpur  2014-01-05        24.42             91.38       6.83


In [29]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 3.127887,
    "longitude": 101.594490,  #PJ 3.127887, 101.594490.
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "PetalingJaya"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/PetalingJaya.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 3.13°N 101.61°E
Elevation: 42.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/PetalingJaya.csv
 Data shape: (4018, 6)
    Country        Region        Date  Temperature  RelativeHumidity  \
1  Malaysia  PetalingJaya  2014-01-01        25.83             82.71   
2  Malaysia  PetalingJaya  2014-01-02        25.99             85.60   
3  Malaysia  PetalingJaya  2014-01-03        24.68             93.15   
4  Malaysia  PetalingJaya  2014-01-04        24.51             93.21   
5  Malaysia  PetalingJaya  2014-01-05        24.69             91.36   

   WindSpeed  
1       8.40  
2       8.09  
3       9.89  
4      11.18  
5       6.83  


In [30]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 6.350000,
    "longitude": 99.800003,  #Langkawi 6.350000, 99.800003..
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "Langkawi"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/#Langkawi.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 6.36°N 99.80°E
Elevation: 47.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/#Langkawi.csv
 Data shape: (4018, 6)
    Country    Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  Langkawi  2014-01-01        26.52             74.52      26.18
2  Malaysia  Langkawi  2014-01-02        27.07             73.66      24.36
3  Malaysia  Langkawi  2014-01-03        27.53             70.03      19.35
4  Malaysia  Langkawi  2014-01-04        27.24             71.30      16.87
5  Malaysia  Langkawi  2014-01-05        27.09             72.04      19.65


In [33]:
# --- SETUP OPEN-METEO CLIENT ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# --- DISPLAY SETTINGS ---
# Optional: ensures all printed numbers show 2 decimal places
pd.options.display.float_format = "{:.2f}".format

# --- API REQUEST SETTINGS ---
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 2.7297,
    "longitude": 101.9381,  #Seremban 2.7297, 101.9381
    "start_date": "2014-01-01",
    "end_date": "2024-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
    "timezone": "Asia/Singapore",
}

# --- FETCH DATA ---
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation():.2f} m")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")

# --- PROCESS HOURLY DATA ---
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_humidity = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "relative_humidity_2m": hourly_humidity,
    #"precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed
}

hourly_df = pd.DataFrame(hourly_data)

# --- CONVERT HOURLY → DAILY ---
hourly_df["date"] = pd.to_datetime(hourly_df["date"])
hourly_df = hourly_df.set_index("date")

daily_df = hourly_df.resample("D").agg({
    "temperature_2m": "mean",
    "relative_humidity_2m": "mean",
    #"precipitation": "sum",   # total daily rainfall
    "wind_speed_10m": "max"   # daily max wind
}).reset_index()

daily_df["date"] = daily_df["date"].dt.date
daily_df = daily_df[daily_df["date"] >= pd.to_datetime("2014-01-01").date()]


# --- ROUND ALL NUMERIC VALUES TO 2 DECIMAL PLACES ---
daily_df["Country"] = "Malaysia"
daily_df["Region"] = "Langkawi"
daily_df = daily_df.round(2)

daily_df = daily_df.rename(columns={
    "date": "Date",
    "temperature_2m": "Temperature",
    "relative_humidity_2m": "RelativeHumidity",
    # "precipitation": "Precipitation",
    "wind_speed_10m": "WindSpeed",
})
daily_df = daily_df[[
    "Country", "Region", "Date",
    "Temperature", "RelativeHumidity",
    "WindSpeed"
]]


# --- SAVE TO GOOGLE DRIVE ---
os.makedirs("/content/drive/MyDrive/Weather", exist_ok=True)
file_path = "/content/drive/MyDrive/Weather/Langkawi1.csv"
daily_df.to_csv(file_path, index=False, float_format="%.2f")

print(f"\n File saved successfully: {file_path}")
print(f" Data shape: {daily_df.shape}")
print(daily_df.head())


Coordinates: 2.71°N 101.92°E
Elevation: 58.00 m
Timezone: b'Asia/Singapore' (b'GMT+8')

 File saved successfully: /content/drive/MyDrive/Weather/Langkawi1.csv
 Data shape: (4018, 6)
    Country    Region        Date  Temperature  RelativeHumidity  WindSpeed
1  Malaysia  Langkawi  2014-01-01        25.61             82.69      12.48
2  Malaysia  Langkawi  2014-01-02        25.81             84.30      11.72
3  Malaysia  Langkawi  2014-01-03        24.86             91.54      10.37
4  Malaysia  Langkawi  2014-01-04        24.65             91.93       9.47
5  Malaysia  Langkawi  2014-01-05        25.00             88.85       6.19
